In [130]:
# selenium for mac
# headless mode OK

import requests
from lxml import html
import bs4
import selenium
from selenium import webdriver
import re

# headless
from selenium.webdriver.chrome.options import Options
chrome_options = Options() # 啟動無頭模式
chrome_options.add_argument('--headless')  #規避google bug
chrome_options.add_argument('--disable-gpu')

# 指定 chromedriver location, create browser object, add headless option
chromedriver = '/usr/local/bin/chromedriver'
# chromedriver = "C:\webdriver\chromedriver.exe"
browser = webdriver.Chrome(chromedriver, options=chrome_options)

#positions = ['0050','006208','00692','1216','1402','2317','2832','2887','2888']
#url = 'https://goodinfo.tw/StockInfo/StockDividendSchedule.asp?STOCK_ID=1216'
#div_url = 'https://goodinfo.tw/StockInfo/StockDividendSchedule.asp?STOCK_ID='
#browser.get(url)
#htmltext = browser.page_source


# 將 result 丟給 beautifulSoup 分析
# display result
#soup = bs4.BeautifulSoup(htmltext,'lxml')
#print(soup.prettify())

# browser.close()

In [131]:

positions

'0050,006208,00692,1216,1402,2317,2823,2887,2888'

In [86]:
# 對應 text
text_all =[
"股利發放年度",
"股利所屬盈餘期間",
"股東會日期",
"除息交易日",  # 除息日程
"除息參考價", 
"填息完成日", 
"填息花費日數", 
"現金股利發放日", 
"除權交易日", # 除權日程
"除權參考價", 
"填權完成日", 
"填權花費日數", 
"現金股利盈餘",
"現金股利公積",
"現金股利合計",
"股票股利盈餘",
'股票股利公積',
"股票股利合計",
"股利合計"]


def get_div(positions):
    # input position
    #positions = '0050,006208,00692,1216,1402,2317,2832,2887,2888'
    positions = positions.split(',')
    list_dict_info = []  # dict of stock
    for pos in positions:
        try:
            div_url = 'https://goodinfo.tw/StockInfo/StockDividendSchedule.asp?STOCK_ID='
            browser.get(div_url+pos)
            htmltext = browser.page_source
            soup = bs4.BeautifulSoup(htmltext,'lxml')
            
            # search for title
            pattern = r'\((\d+)\) (\w*) '
            title_text = soup.title.text[:]
            title_tuple = re.findall(pattern,title_text)[0]
            dict_info = {}
            
            # 內容
            contents = soup.select('#divDetail')[0].select('#row0')[0].select('td')
            dict_info = {title:value.text for title, value in zip(text_all,contents)}
            
            # append ticker/name
            dict_info['ticker'] = title_tuple[0]  # 'ticker' = '0050'
            dict_info['name'] = title_tuple[1] # 'name' = '統一'
            
            list_dict_info.append(dict_info)
            
        except:
            print(f'查尋 {pos} 時發生錯誤')
        
    
    return list_dict_info

In [91]:
list_dict_info[0]['股利發放年度']

'2020'

In [129]:
for item in list_dict_info:
    ticker_name = f"[{item['ticker']}/{item['name']}]"
    ticker_name = ticker_name+ chr(12288)*(16-len(ticker_name))
    item['除息交易日']= '20'+item['除息交易日'].replace(r"'",r'/') if item['除息交易日']!='' else item['除息交易日']
    print(f"【{item['股利發放年度']}】{ticker_name} 股利合計 {item['股利合計']}   除息交易日 {item['除息交易日']}")

【2020】[0050/元大台灣50]　　　 股利合計 2.9； 除息交易日 2020/01/31
【2020】[006208/富邦台50]　　 股利合計 0.48； 除息交易日 2020/07/21即將除息
【2020】[00692/富邦公司治理]　　 股利合計 0.23； 除息交易日 2020/07/21即將除息
【2020】[1216/統一]　　　　　　　 股利合計 2.5； 除息交易日 2020/08/06即將除息
【2020】[1402/遠東新]　　　　　　 股利合計 1.5； 除息交易日 2020/07/27即將除息
【2020】[2317/鴻海]　　　　　　　 股利合計 4.2； 除息交易日 2020/07/23即將除息
【2020】[2832/台產]　　　　　　　 股利合計 1； 除息交易日 2020/06/12
【2020】[2887/台新金]　　　　　　 股利合計 0.8； 除息交易日 
【2020】[2888/新光金]　　　　　　 股利合計 0.4； 除息交易日 


In [88]:
list_dict_info

[{'股利發放年度': '2020',
  '股利所屬盈餘期間': '2020全年',
  '股東會日期': '',
  '除息交易日': "20'01/31",
  '除息參考價': '89.25',
  '填息完成日': "20'02/06",
  '填息花費日數': '5',
  '現金股利發放日': '',
  '除權交易日': '',
  '除權參考價': '',
  '填權完成日': '',
  '填權花費日數': '',
  '現金股利盈餘': '2.9',
  '現金股利公積': '0',
  '現金股利合計': '2.9',
  '股票股利盈餘': '0',
  '股票股利公積': '0',
  '股票股利合計': '0',
  '股利合計': '2.9',
  'ticker': '0050',
  'name': '元大台灣50'},
 {'股利發放年度': '2020',
  '股利所屬盈餘期間': '2020全年',
  '股東會日期': '',
  '除息交易日': "20'07/21即將除息",
  '除息參考價': '',
  '填息完成日': '',
  '填息花費日數': '',
  '現金股利發放日': '',
  '除權交易日': '',
  '除權參考價': '',
  '填權完成日': '',
  '填權花費日數': '',
  '現金股利盈餘': '0.48',
  '現金股利公積': '0',
  '現金股利合計': '0.48',
  '股票股利盈餘': '0',
  '股票股利公積': '0',
  '股票股利合計': '0',
  '股利合計': '0.48',
  'ticker': '006208',
  'name': '富邦台50'},
 {'股利發放年度': '2020',
  '股利所屬盈餘期間': '2020全年',
  '股東會日期': '',
  '除息交易日': "20'07/21即將除息",
  '除息參考價': '',
  '填息完成日': '',
  '填息花費日數': '',
  '現金股利發放日': '',
  '除權交易日': '',
  '除權參考價': '',
  '填權完成日': '',
  '填權花費日數': '',
  '現金股利盈餘': '0.23',
  '現金股利公

In [132]:
positions = '0050,2330,2317,2454,3008,1310,2412,1303,2891,2882,1216'
list_dict_info = get_div(positions)

In [133]:
for item in list_dict_info:
    ticker_name = f"[{item['ticker']}/{item['name']}]"
    ticker_name = ticker_name+ chr(12288)*(16-len(ticker_name))
    item['除息交易日']= '20'+item['除息交易日'].replace(r"'",r'/') if item['除息交易日']!='' else item['除息交易日']
    print(f"【{item['股利發放年度']}】{ticker_name} 股利合計 {item['股利合計']}   除息交易日 {item['除息交易日']}")

【2020】[0050/元大台灣50]　　　 股利合計 2.9   除息交易日 2020/01/31
【2020】[006208/富邦台50]　　 股利合計 0.48   除息交易日 2020/07/21即將除息
【2020】[00692/富邦公司治理]　　 股利合計 0.23   除息交易日 2020/07/21即將除息
【2020】[1216/統一]　　　　　　　 股利合計 2.5   除息交易日 2020/08/06即將除息
【2020】[1402/遠東新]　　　　　　 股利合計 1.5   除息交易日 2020/07/27即將除息
【2020】[2317/鴻海]　　　　　　　 股利合計 4.2   除息交易日 2020/07/23即將除息
【2020】[2832/台產]　　　　　　　 股利合計 1   除息交易日 2020/06/12
【2020】[2887/台新金]　　　　　　 股利合計 0.8   除息交易日 
【2020】[2888/新光金]　　　　　　 股利合計 0.4   除息交易日 


In [69]:
title_tuple

[('1216', '統一')]

In [ ]:
#positions = '0050,006208,00692,1216,1402,2317,2823,2887,2888'

In [70]:
pattern = r'\((\d+)\) (\w*) '
title_text = soup.title.text[:]
title_tuple = re.findall(pattern,title_text)[0]
dict_info['']=title_tuple[1]

In [42]:
contents = soup.select('#divDetail')[0].select('#row0')[0].select('td')
len(contents)

19

In [66]:
dict_info = {title:value.text for title, value in zip(text_all,contents)}

In [71]:
dict_info

{'股利發放年度': '2020',
 '股利所屬盈餘期間': '2019全年',
 '股東會日期': "20'06/24",
 '除息交易日': "20'08/06即將除息",
 '除息參考價': '',
 '填息完成日': '',
 '填息花費日數': '',
 '現金股利發放日': "20'09/11即將發放",
 '除權交易日': '',
 '除權參考價': '',
 '填權完成日': '',
 '填權花費日數': '',
 '現金股利盈餘': '2.5',
 '現金股利公積': '0',
 '現金股利合計': '2.5',
 '股票股利盈餘': '0',
 '股票股利公積': '0',
 '股票股利合計': '0',
 '股利合計': '2.5',
 '股票名稱': '統一'}

In [40]:
text_all =[
"股利發放年度",
"股利所屬盈餘期間",
"股東會日期",
"除息交易日",  # 除息日程
"除息參考價", 
"填息完成日", 
"填息花費日數", 
"現金股利發放日", 
"除權交易日", # 除權日程
"除權參考價", 
"填權完成日", 
"填權花費日數", 
"現金股利盈餘",
"現金股利公積",
"現金股利合計",
"股票股利盈餘",
'股票股利公積',
"股票股利合計",
"股利合計"]

In [41]:
len(text_all)

19

In [36]:
title_text = soup.select('#divDetail')[0].select('thead')[0].select('td')
for item in title_text:
    print(item.text)

股利發放年度
股利所屬盈餘期間
股東會日期
除息日程
除權日程
股東股利 (元/股)
除息交易日
除息參考價
填息完成日
填息花費日數
現金股利發放日
除權交易日
除權參考價
填權完成日
填權花費日數
現金股利
股票股利
股利合計
盈餘
公積
合計
盈餘
公積
合計
